<a href="https://colab.research.google.com/github/mingeunjo/mini_game/blob/main/07_15_%EC%98%81%ED%99%94%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EC%A0%9C%EC%B6%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 라이브러리 불러오기
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report # 혼동행렬 설치 프로그램

from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도 측정하는 코드  - 협업도

In [ ]:
# 1. 나눔 폰트 설치
!sudo apt-get install -y fonts-nanum

# 2. 폰트 캐시 재생성
!sudo fc-cache -fv

# 3. matplotlib 캐시 삭제
!rm ~/.cache/matplotlib -rf

print("설치 완료! ctrl + M .")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,047 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126281 files and dire

In [ ]:
# 코랩 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
url = ('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/' +
       'Online%20Retail.xlsx')
df = pd.read_excel(url)
print(df.shape)
df.head(10)

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
df.rename(columns={"Description": "customerID"}, inplace=True)

In [ ]:
df = df[df['Quantity'] > 0]
# 수량이 양수인 경우만 저장

df = df.dropna(subset=['CustomerID'])

customer_item_matrix = df.pivot_table(index='CustomerID',
columns='StockCode', values='Quantity', aggfunc=sum)
# 평균값은 aggfunc=sum 대신 aggfunc='mean' 사용

In [ ]:
df = df.dropna(subset=['CustomerID'])
# 컬러이 업슨ㄴ 경우 삭제 , 삭제후 남은 행 수 확인

In [ ]:
customer_item_matrix.loc[12481:].head(10)

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0
12483.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
12484.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
12488.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
12489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
12490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0
12491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
12492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
12493.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0


In [ ]:
# 사용자-아이템 행렬을 만든다(applymap 함수를 이용해서 구매했으면 1, 구매하지 않았으면 0으로 인코딩)
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix.iloc[12481:].head(20)

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,


In [ ]:
# 사용자간 코사인 유사성 행렬을 정의
from sklearn.metrics.pairwise import cosine_similarity

user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_user_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,4329,4330,4331,4332,4333,4334,4335,4336,4337,4338
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
2,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
3,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
4,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [ ]:
user_user_sim_matrix.columns = customer_item_matrix.index
user_user_sim_matrix['CustomerID'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')
user_user_sim_matrix

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044866,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.062318,0.000000,0.000000,0.000000,0.000000,0.0,0.041523,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.105409,1.000000,0.119523,0.000000,0.000000,0.000000
18281.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.049629,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.119523,1.000000,0.000000,0.046613,0.000000
18282.0,0.0,0.113776,0.000000,0.067574,0.000000,0.037582,0.0,0.000000,0.160128,0.079305,...,0.0,0.174078,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.017800,0.000000


In [ ]:
# CustomerID = 12350에 대해 사용자간 유사성 지수를 내림차순으로 프린트함
user_user_sim_matrix.loc[12350.0].sort_values(ascending=False)

,12350.0
CustomerID,
12350.0,1.000000
17935.0,0.183340
12414.0,0.181902
12652.0,0.175035
16692.0,0.171499
...,...
15437.0,0.000000
15438.0,0.000000
12820.0,0.000000


In [ ]:
# 아이템간 유사성 행렬 정의
# customer_item_matrix.T 점 T는 인덱스 부분을 바꿔준다.
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
item_item_sim_matrix.columns = customer_item_matrix.T.index
item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


In [ ]:
top10 = list(item_item_sim_matrix.loc[10120].sort_values(ascending=False).head(10).index)

In [ ]:
top10

[10120, 16237, 16236, '84206A', 16238, 84199, '16258A', 21722, 20986, 84466]

In [ ]:
# 결측치 제거
df.dropna(subset=['CustomerID', 'Description'], inplace=True)

In [ ]:
df.shape

(406829, 9)

In [ ]:
# 총 구매 금액 컬럼 추가
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 피벗 테이블 (사용자-상품)
user_item = df.pivot_table(index='CustomerID', columns='Description', values='Quantity', aggfunc='sum').fillna(0)


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18282.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 유사도 계산
user_similarity = cosine_similarity(user_item)
user_sim_df = pd.DataFrame(user_similarity, index=user_item.index, columns=user_item.index)
user_sim_df

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.148879,0.020750,0.014435,0.034833,0.0,0.022843,0.506252,0.186107,...,0.0,0.0,0.407060,-0.001245,0.015133,0.037236,0.000000,0.011921,0.075245,0.108942
12348.0,0.0,0.148879,1.000000,0.000169,0.000315,0.001578,0.0,0.010634,0.286226,0.226244,...,0.0,0.0,0.168758,0.000000,0.000000,0.000000,0.000000,0.000000,0.177440,0.110096
12349.0,0.0,0.020750,0.000169,1.000000,0.030121,0.072258,0.0,0.004931,0.000180,0.150819,...,0.0,0.0,0.000000,-0.000344,0.015680,0.000000,0.000000,0.014689,0.038343,0.005644
12350.0,0.0,0.014435,0.000315,0.030121,1.000000,0.001938,0.0,0.000000,0.000000,0.001179,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019636,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.037236,0.000000,0.000000,0.000000,0.000000,0.0,0.002707,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.043042,1.000000,0.098363,0.000000,0.000000,0.000000
18281.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000435,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.098363,1.000000,0.000000,0.067794,0.000000
18282.0,0.0,0.011921,0.000000,0.014689,0.000000,0.002966,0.0,0.000000,0.007169,0.004772,...,0.0,0.0,0.000000,-0.001372,0.000000,0.000000,0.000000,1.000000,0.003825,0.000000


In [ ]:
# 예: 사용자 17850과 유사한 사용자들이 구매한 상품 Top10 추천
target_user = 17850
similar_users = user_sim_df[target_user].sort_values(ascending=False).iloc[1:6]  # 자기 자신 제외, use iloc for positional slicing
similar_users

,17850.0
CustomerID,
13161.0,0.399152
13831.0,0.356111
14209.0,0.314731
15636.0,0.311830
14440.0,0.310954


In [ ]:
# 유사한 사용자의 구매 상품 가져오기
recommendations = user_item.loc[similar_users.index].mean().sort_values(ascending=False).head(10)
recommendations

,0
Description,
WHITE HANGING HEART T-LIGHT HOLDER,114.8
WOODEN FRAME ANTIQUE WHITE,31.2
WOODEN PICTURE FRAME WHITE FINISH,31.2
RED HANGING HEART T-LIGHT HOLDER,20.0
T-LIGHT HOLDER SWEETHEART HANGING,11.2
HANGING CLEAR MINI BOTTLE,9.6
WORLD WAR 2 GLIDERS ASSTD DESIGNS,9.6
HOME BUILDING BLOCK WORD,9.6
DECORATIVE WICKER HEART SMALL,9.6


In [ ]:
print("사용자 기반 추천 Top 10 상품:")
recommendations

사용자 기반 추천 Top 10 상품:


,0
Description,
WHITE HANGING HEART T-LIGHT HOLDER,114.8
WOODEN FRAME ANTIQUE WHITE,31.2
WOODEN PICTURE FRAME WHITE FINISH,31.2
RED HANGING HEART T-LIGHT HOLDER,20.0
T-LIGHT HOLDER SWEETHEART HANGING,11.2
HANGING CLEAR MINI BOTTLE,9.6
WORLD WAR 2 GLIDERS ASSTD DESIGNS,9.6
HOME BUILDING BLOCK WORD,9.6
DECORATIVE WICKER HEART SMALL,9.6


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 상품 설명 중복 제거
products = df[['Description']].drop_duplicates().reset_index(drop=True)
products

,Description
0,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART.
...,...
3891,"LETTER ""W"" BLING KEY RING"
3892,"LETTER ""Z"" BLING KEY RING"
3893,PINK CRYSTAL SKULL PHONE CHARM
3894,CREAM HANGING HEART T-LIGHT HOLDER


In [ ]:
# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(products['Description'])
tfidf_matrix.shape

(3896, 1979)

In [ ]:
# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.21657896, 0.        , ..., 0.        , 0.77004192,
        0.        ],
       [0.21657896, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.18540875,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.77004192, 0.        , 0.18540875, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
print("콘텐츠 기반 추천 Top 10 상품:")
products.iloc[top_indices]['Description']

콘텐츠 기반 추천 Top 10 상품:


,Description
3759,PINK HANGING HEART T-LIGHT HOLDER
58,RED HANGING HEART T-LIGHT HOLDER
3894,CREAM HANGING HEART T-LIGHT HOLDER
167,HANGING HEART ZINC T-LIGHT HOLDER
505,HEART T-LIGHT HOLDER
1928,HEART T-LIGHT HOLDER
1582,HANGING HEART JAR T-LIGHT HOLDER
310,SILVER HANGING T-LIGHT HOLDER
1042,GLASS HEART T-LIGHT HOLDER
3700,HANGING BUTTERFLY T-LIGHT HOLDER


# 개인 과제 = 영화데이터

In [ ]:
data = ('https://raw.githubusercontent.com/sd12832/MoViZ/master/' +
        'src/tmdb_5000_credits.csv')
movie1 = pd.read_csv(data)
print(movie1.shape)
movie1.head()

(4803, 4)


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
url2 = ('https://raw.githubusercontent.com/sd12832/MoViZ/master/' +
        'src/tmdb_5000_movies.csv')
movie2 = pd.read_csv(url2)
print(movie2.shape)
movie2.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
movie1.columns = ['id', 'title', 'cast', 'crew']
movie1.head()

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movie = movie2.merge(movie1, on='id')
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,tagline,title_x,vote_average,vote_count,title_y,cast_x,crew_x,title,cast_y,crew_y
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movie1.shape

(4803, 4)

In [ ]:
movie2.shape # 행 4803 movie1,moive2 행이 맞는지 확안해 준다.

(4803, 23)

In [ ]:
# 평점 통계를 계산
C = movie['vote_average'].mean()
m = movie['vote_count'].quantile(0.9)
C,m

(np.float64(6.092171559442016), np.float64(1838.4000000000015))

In [ ]:
movies = movie.loc[movie['vote_count']>=m]
movies.shape

(481, 26)

In [ ]:
# IMDB 평점 계산 함수
def ration(x, m=m, C=C):
  v = x['vote_count']
  R = x['vote_average']
  return (v / (v + m) * R) + (m / (m + v) * C)

# movies 데이터프레임에 IMDB 평점 계산 함수 적용
score = movies.apply(ration, axis=1)
movies = movies.assign(score = score)
movies = movies.sort_values('score', ascending=False)
movies[['title' , 'vote_count', 'vote_average', 'score','original_language']].head(10)

,title,vote_count,vote_average,score,original_language
1881,The Shawshank Redemption,8205,8.5,8.059258,en
662,Fight Club,9413,8.3,7.939256,en
65,The Dark Knight,12002,8.2,7.920020,en
3232,Pulp Fiction,8428,8.3,7.904645,en
96,Inception,13752,8.1,7.863239,en
3337,The Godfather,5893,8.4,7.851236,en
95,Interstellar,10867,8.1,7.809479,en
809,Forrest Gump,7927,8.2,7.803188,en
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243,en
1990,The Empire Strikes Back,5879,8.2,7.697884,en


In [ ]:
# 비영어권 영화 저장
movies2 = movie[movie['original_language'] != 'en']
movies2.shape

(298, 26)

In [ ]:
# 평점 통계를 계산 - 비영화권
C = movies2['vote_average'].mean()
m = movies2['vote_count'].quantile(0.5)
C,m

(np.float64(6.492617449664429), np.float64(90.5))

In [ ]:
# 가중 평점 계산 함수 적용
score = movies2.apply(ration, axis=1)

# score 컬럼 추가
f_movies = movies2.assign(score=score)

# score 기준 정렬 (주의: movies2가 아닌 f_movies 기준으로!)
f_movies = f_movies.sort_values('score', ascending=False)

# 상위 10개 영화 출력
f_movies[['title', 'vote_count', 'vote_average', 'score', 'original_language']].head(10)


,title,vote_count,vote_average,score,original_language
2294,Spirited Away,3840,8.3,7.585209,ja
4302,"The Good, the Bad and the Ugly",2311,8.1,7.210428,it
1260,Amélie,3310,7.8,7.190166,fr
1987,Howl's Moving Castle,1991,8.2,7.188084,ja
2247,Princess Mononoke,1983,8.2,7.185965,ja
3866,City of God,1814,8.1,7.089379,pt
3940,Oldboy,1945,8.0,7.072963,ko
2465,Pan's Labyrinth,3041,7.6,7.031899,es
3622,Once Upon a Time in the West,1128,8.1,6.855666,it
4535,Seven Samurai,878,8.2,6.773468,ja


In [ ]:
movies2['overview'].loc[3940] # 올드보이 줄거리

'With no clue how he came to be imprisoned, drugged and tortured for 15 years, a desperate businessman seeks revenge on his captors.'

In [ ]:
# prompt: 영어 불용어 제거

# 비영어권 영화 필터링 및 추천
movies2 = movie[movie['original_language'] != 'en']

# 평점 통계 계산 (비영어권 영화 기준)
C = movies2['vote_average'].mean()
m = movies2['vote_count'].quantile(0.5)

# 가중 평점 계산 함수 적용
score = movies2.apply(ration, axis=1)

# score 컬럼 추가
f_movies = movies2.assign(score=score)

# score 기준 정렬
f_movies = f_movies.sort_values('score', ascending=False)

# 상위 10개 비영어권 영화 출력
f_movies[['title', 'vote_count', 'vote_average', 'score', 'original_language']]

,title,vote_count,vote_average,score,original_language
2294,Spirited Away,3840,8.3,7.585209,ja
4302,"The Good, the Bad and the Ugly",2311,8.1,7.210428,it
1260,Amélie,3310,7.8,7.190166,fr
1987,Howl's Moving Castle,1991,8.2,7.188084,ja
2247,Princess Mononoke,1983,8.2,7.185965,ja
...,...,...,...,...,...
4257,#Horror,52,3.3,6.015366,de
4586,What Happens in Vegas,923,5.8,5.994513,da
1518,Dragon Wars: D-War,104,4.0,5.980152,ko
4308,Desire,140,4.1,5.951197,fr


# 줄거리 기반 추천 알고리즘


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. overview 컬럼의 결측값을 빈 문자열로 대체
movies2['overview'] = movies2['overview'].fillna('')

# 2. TF-IDF 벡터화 객체 생성
tfidf = TfidfVectorizer(stop_words='english')  # 불용어 제거 포함

# 3. TF-IDF 행렬 생성 (희소 행렬 형태)
tfidf_matrix = tfidf.fit_transform(movies2['overview'])

# 4. 결과 확인 (문서 수 × 단어 수)
print("TF-IDF 행렬의 크기:", tfidf_matrix.shape)

TF-IDF 행렬의 크기: (298, 4392)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 결측값 처리
movies2['overview'] = movies2['overview'].fillna('')

# TF-IDF 벡터화(텍스트 벡터화) : 자주 등장하지만 흔하지 않은 단어에 높은 가중치 부여
tfidf = TfidfVectorizer(stop_words='english') # 불용어제거
tfidf_matrix = tfidf.fit_transform(movies2['overview'])

# 코사인 유사도 계산 : 영화 간 줄거리 유사도를 코사인 유사도로 측정
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화 제목과 인덱스 매핑 (행 위치 기반으로 매핑)
indices = pd.Series(data=range(len(movies2)), index=movies2['title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movies2['title'].iloc[movie_indices]


In [ ]:
sim_matrix = pd.DataFrame(
    cosine_similarity(tfidf_matrix),  # cosine similarity 계산
    index=movies2['title'],           # 행 인덱스: 영화 제목
    columns=movies2['title']          # 열 인덱스: 영화 제목
)

# 예시: 특정 영화 기준으로 유사도 높은 Top 10 추출
movie_name = 'Sympathy for Lady Vengeance'  # 기준 영화명
top_10_similar_items = sim_matrix[movie_name].sort_values(ascending=False).iloc[1:11]
top_10_similar_items

,Sympathy for Lady Vengeance
title,
Savva. Heart of the Warrior,0.171887
Once Upon a Time in the West,0.153966
CJ7,0.116383
Ong Bak 2,0.086192
Oldboy,0.081390
"Good bye, Lenin!",0.078604
"I Love You, Don't Touch Me!",0.075305
Irreversible,0.071913
Hero,0.067534


# 메타데이터 추천 알고리즘

In [ ]:
# 데이터 정리: 문자열 통일 & 소문자 처리 : 리스트 또는 문자열을 받아 공백 제거 + 소문자 변환
def clean_data(x):
    if isinstance(x, list):
        return ' '.join(x)
    else:
        return str(x)
for feature in ['cast_x', 'crew_x', 'keywords', 'genres']:
    movie[feature] = movie[feature].apply(clean_data)


In [ ]:
# 각 컬럼 정제 및 soup 생성 : 여러 메타데이터를 한 덩어리 텍스트(soup) 로 만듦
movies2['soup'] = movie['cast_x'] + ' ' + movie['crew_x'] + ' ' + movie['keywords'] + ' ' + movie['genres']

In [ ]:
# 텍스트 → 수치 벡터화 (CountVectorizer)
# CountVectorizer: 단어 빈도 기반으로 각 영화의 soup을 벡터로 변환
# stop_words='english': 불용어 제거 (a, the, is 등)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movies2['soup'])

# 영화 간 유사도 계산 (Cosine Similarity) : 벡터끼리의 유사도 계산 → 영화 간 유사도 행렬 생성 (NxN)
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# 인덱스 → 영화제목 매핑 : 영화 제목을 주면 해당 영화의 행 인덱스를 쉽게 찾도록 설정
indices = pd.Series(range(len(movies2)), index=movies2['title']).drop_duplicates()

In [ ]:
# 10개 영화 추천 함수
# 해당 영화와 유사한 영화 10개를 유사도 기준으로 정렬하여 리턴
# reverse=True: 유사도 높은 순 정렬
# [1:11]: 자기 자신은 제외하고 상위 10개
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return movies2['title'].iloc[movie_indices]

In [ ]:
# 결과 사용
get_recommendations("Biutiful")

,title
2378,The White Ribbon
3206,Polisse
3562,Instructions Not Included
2508,Wasabi
985,The Messenger: The Story of Joan of Arc
861,A Very Long Engagement
3995,The Dress
4535,Seven Samurai
3330,The Wave
2271,Ramanujan
